In [ ]:
import torch
from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from transformers import set_seed
from transformers import AutoTokenizer
from peft import prepare_model_for_kbit_training, get_peft_model
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModelForCausalLM, PeftModel, PeftConfig
import torch
from datasets import DatasetDict, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import gc
import json


In [ ]:
seed = 42
set_seed(seed)

In [ ]:
bnb_configs = []

for b in [False,True]:
    for e in [False,True]:
        for f in [ "float32","bfloat16"]:
            bnb_configs.append(
                {
                "load_in_4bit":b,
                "bnb_4bit_use_double_quant":e,
                "bnb_4bit_compute_dtype":f 
                }
            )

len(bnb_configs)

In [ ]:
lora_configs = []

for a in [1024,64,8,1]:
    for b in [1024,64,8,1]:
        for d in [0.1,0]:
            lora_configs.append(
                {
                    "r":a,
                    "lora_alpha":b,
                    "lora_dropout":d,
                }
            )
                            
len(lora_configs)

In [ ]:
base_configs = []

for c in [0.1,0]:
    for d in [1e-5,1e-4]:
        for f in [False,True]:
                base_configs.append(
                    {
                        "weight_decay":c,
                        "learning_rate":d,
                        "fp16":f,
                    }
                )

len(base_configs)

In [ ]:
train_configs = []

for a in [True, False]:
    for b in [False,True]:
        if not a and b: continue
        train_configs.append(
            {
                "lora":a,
                "bnb":b
            }
        )

len(train_configs)

In [ ]:
def write_json(new_data, filename='data_incomplete.json'):
    with open(filename,'r+') as file:
        file_data = json.load(file)
        file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file, indent = 4)

In [ ]:
def create_pipeline_and_print(model,tokenizer,p=False,t=""):
    prompt = "Hello"
    gen = pipeline(model=model, tokenizer=tokenizer, task="text-generation", device_map="auto",max_new_tokens=10)
    result = gen(prompt)[0]["generated_text"]
    if p: print(t, ": ",result)
    
    del gen
    gc.collect()
    
    return result


In [ ]:
i = 0

def train_model(base_config, lora_config, bnb_config, save=True, prt=False):
    global i
    i += 1
    if i < 28: return
    
    model_name = "TinyLlama/TinyLlama_v1.1"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    data = Dataset.from_json("data/banana.json")
    data = data.map(lambda example : tokenizer(example["text"], return_tensors="np"), batched=True)

    if bnb_config:
        bnb = BitsAndBytesConfig(
            load_in_4bit=bnb_config["load_in_4bit"],
            bnb_4bit_use_double_quant=bnb_config["bnb_4bit_use_double_quant"],
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=bnb_config["bnb_4bit_compute_dtype"]
        )
        
        model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map="auto",
                    revision="main",
                    quantization_config = bnb
                )
    else:
        model = AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto",
                revision="main"
            )
    
    if lora_config:
        lora = LoraConfig(
            r = lora_config["r"],
            lora_alpha = lora_config["lora_alpha"],
            init_lora_weights = True,
            lora_dropout = lora_config["lora_dropout"],
            bias = 'none',
            task_type="CAUSAL_LM"
        )
        
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model,lora)
    
    args = TrainingArguments(
        output_dir=".",
        fp16=base_config["fp16"],
        weight_decay=base_config["weight_decay"],
        learning_rate=base_config["learning_rate"],
        label_names=['input_ids'],
        num_train_epochs=100,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=1,
        no_cuda=False,
        optim="paged_adamw_8bit",
        eval_steps=1000
    )
    
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=data,
        tokenizer=tokenizer,
        data_collator=collator
    )
    
    trainer.train()
    save_path = "tmp"
    model.save_pretrained(save_path)


    generation = {"args":base_config,"lora":lora_config,"bnb":bnb_config}

    base_model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=False, torch_dtype="bfloat16")
    
    generation["base"] = create_pipeline_and_print(model=base_model,tokenizer=tokenizer,t="base",p=prt)
    
    generation["direct"] = create_pipeline_and_print(model=model,tokenizer=tokenizer,t="direct",p=prt)
    
    if lora_config:
        merged_model = model.merge_and_unload()

        generation["merged"] = create_pipeline_and_print(model=merged_model,tokenizer=tokenizer,t="merged",p=prt)
    
    
    if lora_config:
        loaded_model = PeftModelForCausalLM.from_pretrained(base_model,save_path)
    else:
        loaded_model = AutoModelForCausalLM.from_pretrained(save_path)
    
    generation["loaded"] = create_pipeline_and_print(model=loaded_model,tokenizer=tokenizer,t="loaded",p=prt)
    
    if lora_config:
        merged_loaded_model = loaded_model.merge_and_unload()
        generation["merged_loaded"] = create_pipeline_and_print(model=merged_loaded_model,tokenizer=tokenizer,t="mergedLoaded",p=prt)
        
        del loaded_model
        del merged_loaded_model
    
    if save: write_json(generation)
    
    del tokenizer
    del collator
    del data
    del args
    del model
    del base_model
    del merged_model
    del generation
    
    gc.collect()

In [ ]:
train_model({"weight_decay":0.1,"learning_rate":0.0001,"fp16":False},
            {"r":1024,"lora_alpha":64,"lora_dropout":0.1},
            {"load_in_4bit":True,"bnb_4bit_use_double_quant":True,"bnb_4bit_compute_dtype":"bfloat16"}, 
            #False,
            prt=True,save=False
            )

In [ ]:
"""bnb = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    revision="main",
    quantization_config = bnb
)


lora = LoraConfig(
    r = 64,
    lora_alpha = 64,
    init_lora_weights = True,
    lora_dropout = 0.1,
    bias = 'none',
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model,lora)
    
args = TrainingArguments(
    output_dir=".",
    fp16=True,
    weight_decay=0.1,
    learning_rate=1e-5,
    label_names=['input_ids'],
    num_train_epochs=100,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    no_cuda=False,
    optim="paged_adamw_8bit",
)
    
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=collator
)

trainer.train()
save_path = "tmp"
trainer.save_model(save_path)"""

In [ ]:
"""create_pipeline_and_print(model)

base_model = AutoModelForCausalLM.from_pretrained(model_name)
create_pipeline_and_print(base_model)

model = model.merge_and_unload()
create_pipeline_and_print(model)
create_pipeline_and_print(base_model)

loaded_model = AutoModelForCausalLM.from_pretrained(save_path)
create_pipeline_and_print(loaded_model)

peft_model = PeftModelForCausalLM.from_pretrained(base_model,save_path)
create_pipeline_and_print(peft_model)

peft_model = peft_model.merge_and_unload()
create_pipeline_and_print(peft_model)"""

In [ ]:
%%time

for train_config in train_configs:
    for base_config in base_configs:
        if train_config["lora"]:
            for lora_config in lora_configs:
                if train_config["bnb"]:
                    for bnb_config in bnb_configs:
                        train_model(base_config, lora_config, bnb_config)
                else:
                    train_model(base_config, lora_config, False)
        else:
            train_model(base_config, False, False)
            